# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

## Import Library

In [1]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

## Import Model

In [2]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant, Chapter
from collections import namedtuple

## Data extraction from model

In [140]:
class LaboratoryData:

    def __init__(self, lab = Laboratory):
        self._lab = lab
        self._laboratories = lab.objects.all()

    @property
    def laboratories(self): 
        return self._laboratories.values()
    
    def get_assistant(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        assistants = laboratory.assistants.all()
        return assistants.values()
    
    def get_module(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        modules = laboratory.modules.all()
        return modules.values()
    
class ModuleData:

    def __init__(self, module = Module):
        self._module = module
        self._modules = module.objects.all()

    @property
    def modules(self): 
        return self._modules.values()
    
    def get_module(self,id):
        module = self._modules.filter(id=id).first()
        return module
    
    def get_module_chapter(self, id):
        module = self.get_module(id)
        return module.chapters.all().values()
    
    def get_group(self, id):
        module = self.get_module(id)
        return module.groups.all().values()
        
class ChapterData:

    def __init__(self, chapter = Chapter):
        self._chapter = chapter
        self._chapters = chapter.objects.all()

    @property
    def chapters(self): 
        return self._chapters.values()
    
    def get_chapter(self,id):
        chapter = self._chapters.filter(id=id).first()
        return chapter
    
    def get_chapter_module(self, id):
        chapter = self.get_chapter(id)
        return chapter.module

class ParticipantData:

    def __init__(self, participant = Participant):
        self._participant = participant
        self._participants = participant.objects.all()
    
    @property
    def participants(self):
        return self._participants.values()
    
    def group(self, id):
        group_memberships = self._participants.filter(id=id).first().group_memberships.all()
        groups = []
        for group_membership in group_memberships:
            group = group_membership.group
            groups.append(group)
        return groups
    
class GroupData:
    def __init__(self, group = Group):
        self._group = group
        self._groups = group.objects.all()

    @property
    def groups(self):
        return self._groups.values()
    
    def get_group(self, id):
        group = self._groups.filter(id=id).first()
        return group
    
    def participants(self, id):
        group = self._groups.filter(id=id).first()
        group_memberships = group.group_memberships.all()
        participants = []
        for group_membership in group_memberships:
            participant = group_membership.participant
            participants.append(participant)
        return participants
    
    def get_participant_schedule(self, id):
        participants = self.participants(id)
        participant_schedule = []
        for participant in participants:
            participant_schedule.append(participant.regular_schedule)
        return participant_schedule
    
    def get_group_schedule(self, id):
        participant_schedule = self.get_participant_schedule(id)
        if len(participant_schedule) == 0:
            return {}
        days = participant_schedule[0].keys()
        merged_schedule = {day: {} for day in days}
        for day in days:
            for time_slot in participant_schedule[0][day]:
                is_available = all(schedule[day][time_slot] for schedule in participant_schedule)
                merged_schedule[day][time_slot] = is_available
        return merged_schedule
                
            
class AssistantData:

    def __init__(self, assistant = Assistant):
        self._assistant = assistant
        self._assistants = assistant.objects.all()

    @property
    def assistants(self):
        return self._assistants.values()

    
    def get_assistant(self, id):
        assistant = self._assistants.filter(id=id).first()
        return assistant
    
    def get_assistant_modules(self,id):
        assistant = self._assistant.filter(id=id).first()
        assistant_memberships = assistant.assistant_memberships.all()
        modules = []
        for assistant_membership in assistant_memberships:
            module = assistant_membership.module
            modules.append(module)
        return modules

## Genetic Algorithm

In [4]:
import pandas as pd

In [141]:
# Assigning data to variables
lab_data = LaboratoryData()
module_data = ModuleData()
participant_data = ParticipantData()
group_data = GroupData()
assistant_data = AssistantData()
chapter_data = ChapterData()

In [144]:
assistant_data.assistants

<QuerySet [{'id': 1, 'name': 'Fatimah', 'nim': '1', 'laboratory_id': 1, 'semester_id': 1, 'regular_schedule': {'Friday': {'Shift1': True, 'Shift2': True, 'Shift3': True, 'Shift4': True, 'Shift5': True, 'Shift6': False}, 'Monday': {'Shift1': True, 'Shift2': True, 'Shift3': True, 'Shift4': True, 'Shift5': True, 'Shift6': False}, 'Tuesday': {'Shift1': True, 'Shift2': True, 'Shift3': True, 'Shift4': True, 'Shift5': True, 'Shift6': False}, 'Saturday': {'Shift1': True, 'Shift2': True, 'Shift3': True, 'Shift4': True, 'Shift5': False, 'Shift6': True}, 'Thursday': {'Shift1': True, 'Shift2': True, 'Shift3': True, 'Shift4': True, 'Shift5': True, 'Shift6': True}, 'Wednesday': {'Shift1': True, 'Shift2': True, 'Shift3': True, 'Shift4': False, 'Shift5': False, 'Shift6': True}}, 'prefered_schedule': {'Friday': {'Shift1': False, 'Shift2': True, 'Shift3': True, 'Shift4': True, 'Shift5': False, 'Shift6': True}, 'Monday': {'Shift1': True, 'Shift2': True, 'Shift3': True, 'Shift4': True, 'Shift5': False, 'S

In [6]:
pd.DataFrame(participant_data.participants)

,id,name,nim,semester_id,regular_schedule
0,5,Setio Ningrum,3332210016,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
1,6,Ruth Anandina,3332210051,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
2,7,ADHITAMA WIRA YUDHA,3332190029,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
3,8,FARHAN NUGROHO,3332190072,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
4,9,BAGAS NURYANTO,3332190080,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
...,...,...,...,...,...
102,55,Maulana Ali Akbar,3332210082,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
103,65,Naga Tunggal,3332210004,1,"{'Friday': {'Shift1': True, 'Shift2': False, '..."
104,86,Randy eleanor,3332210057,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
105,106,MAHESA NURUL VIKAR,3332200087,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."


In [37]:
module_data.get_group(1)

AttributeError: 'QuerySet' object has no attribute 'id'

### Gene Representation for Scheduling Problem

In [8]:
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

In [9]:
# Gene Representation
# 1. Laboratory
# 2. Module
# 3. Module chapter
# 4. Group
# 5. Assistant
# 6. Time Slot : Date, Day, Time
# 7. Availability: json, e.g. {"Friday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Monday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Tuesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Saturday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": false, "Shift6": true}, "Thursday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": true}, "Wednesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": false, "Shift5": false, "Shift6": true}}

class Gene:
    def __init__(self, laboratory, module, module_chapter, group, assistant, time_slot: TimeSlot):
        self.laboratory = laboratory
        self.module = module
        self.module_chapter = module_chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot
        
    def __repr__(self):
        return f"Gene(lab={self.laboratory}, module={self.module}, module_chapter={self.module_chapter}, group={self.group}, assistant={self.assistant}, time_slot={self.time_slot})"
    
    def generate(self):
        return {
            "lab": self.laboratory,
            "module": self.module,
            "module_chapter": self.module_chapter,
            "group": self.group,
            "assistant": self.assistant,
            "time_slot": self.time_slot
        }
    

In [10]:
# Test Gene
gene = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[0]["id"],assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift1"))
gene2 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[1]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift2"))
gene3 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[2]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Monday", "Shift1"))

In [11]:
gene.time_slot == gene3.time_slot

False

In [12]:
group_data.groups

<QuerySet [{'id': 1, 'name': 'PL-1', 'module_id': 1}, {'id': 2, 'name': 'PL-2', 'module_id': 1}, {'id': 3, 'name': 'PL-3', 'module_id': 1}, {'id': 4, 'name': 'PL-4', 'module_id': 1}, {'id': 5, 'name': 'PL-5', 'module_id': 1}, {'id': 6, 'name': 'PL-6', 'module_id': 1}, {'id': 7, 'name': 'PL-7', 'module_id': 1}, {'id': 8, 'name': 'PL-8', 'module_id': 1}, {'id': 9, 'name': 'PL-9', 'module_id': 1}, {'id': 10, 'name': 'PL-10', 'module_id': 1}, {'id': 11, 'name': 'PL-11', 'module_id': 1}, {'id': 12, 'name': 'PL-12', 'module_id': 1}, {'id': 13, 'name': 'PL-13', 'module_id': 1}, {'id': 14, 'name': 'PL-14', 'module_id': 1}, {'id': 15, 'name': 'PL-15', 'module_id': 1}, {'id': 16, 'name': 'PL-16', 'module_id': 1}, {'id': 17, 'name': 'PL-17', 'module_id': 1}, {'id': 18, 'name': 'PL-18', 'module_id': 1}, {'id': 19, 'name': 'PL-19', 'module_id': 1}, {'id': 20, 'name': 'PL-20', 'module_id': 1}, '...(remaining elements truncated)...']>

#### Gene-Level Constraints: Hard Constraints

In [30]:
# Constraint Class for Constraint Function
# 1. Ensure that each chapter is assigned to the correct module.
# 2. Ensure that each module is assigned to the correct lab.
# 3. Ensure that each group is assigned to the correct module.
# 4. Ensure that each assistant is assigned to the correct lab.

class ConstraintChecker:
    def __init__(self, gene, lab_data: LaboratoryData, module_data: ModuleData, group_data: GroupData, assistant_data: AssistantData, chapter_data: ChapterData):
        self.gene = gene
        self.lab_data = lab_data
        self.module_data = module_data
        self.group_data = group_data
        self.assistant_data = assistant_data
        self.chapter_data = chapter_data

    def chapter_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        chapter = self.chapter_data.get_chapter(self.gene.module_chapter)
        return module == chapter.module
    
    def module_lab_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        return module.laboratory.id == self.gene.laboratory
    
    def group_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        group = self.group_data.get_group(self.gene.group)
        return module == group.module
    
    def assistant_lab_constraint(self):
        assistant = self.assistant_data.get_assistant(self.gene.assistant)
        return assistant.laboratory.id == self.gene.laboratory
    
    def check(self):
        return self.chapter_module_constraint() and self.module_lab_constraint() and self.group_module_constraint() and self.assistant_lab_constraint()

In [27]:
assistant_data.get_assistant(1).laboratory.id

1

### Chromosome Representation for Scheduling Problem

In [15]:
# Chromosome Representation
# 1. Gene
# 2. Fitness

class Chromosome:
    def __init__(self, genes, fitness):
        self.genes = genes
        self.fitness = fitness
        
    def __repr__(self):
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def __eq__(self, other):
        return self.genes == other.genes and self.fitness == other.fitness
    
    def __hash__(self):
        return hash((self.genes, self.fitness))
    
    def __str__(self) -> str:
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def generate(self):
        return Chromosome(self.genes, self.fitness)

In [16]:
# test chromosome
chromosome = Chromosome([gene, gene2, gene3], 0.5)

#### Chromosome-Level Constraints

#### Objective Function, 
satisfying hard and soft constraints.
1. Hard Constraints
    - Each course must be assigned to a single room in a single time slot.
    - Each room can only accommodate a specified course at a time.
    - Each lecturer can only teach one course at a time.
    - Each lecturer can only assist one group of students at a time.
    - Each student can only take one course at a time.
    - Each course must be assigned to a single time slot.
    - Each course must be assigned to a single lecturer.
    - Each course must be assigned up to a specified student group.
    - Each course must be finished within a specified time slot.
    - Course held for 5 days a week.
    - No two groups can be assigned to the same lab and time slot simultaneously.
    - Make sure each group is assigned to all the courses they need. In this case, each group must be assigned to each chapter of the course module.

2. Soft Constraints
    - Maximum units that can be held simultaneously on a module the same is 2 modules per week.
    - The period between each unit is at least 2 days.
    - The number of groups for each course in 1 shift is 3 groups

Objective class
1. Minimize conflict: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.
2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

In [19]:
# Objective Class for Fitness Function
# 1. Minimize Conflicts: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

class MinimizeConflicts:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.conflicts = namedtuple("Conflicts", ["time_slot", "lab"])
        self.conflicts_time_slot = set()
        self.assigned_time_slot = set()
        self.conflict_count = 0


    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Conflicts Time Slot: {self.conflicts_time_slot}, Assigned Time Slot: {self.assigned_time_slot}, Conflict Count: {self.conflict_count}"
    
    def __eq__(self, other):
        return self.chromosome == other.chromosome and self.conflicts_time_slot == other.conflicts_time_slot and self.assigned_time_slot == other.assigned_time_slot and self.conflict_count == other.conflict_count
    
    def calculate_conflicts(self):
        for gene in self.chromosome.genes:
            if (gene.laboratory, gene.time_slot) in self.assigned_time_slot:
                self.conflicts_time_slot.add(self.conflicts(gene.time_slot, gene.laboratory))
                self.conflict_count += 1
            else:
                self.assigned_time_slot.add((gene.laboratory, gene.time_slot))
        return self.conflict_count

    def calculate_fitness(self):
        self.calculate_conflicts()
        return self.conflict_count


In [20]:
a = MinimizeConflicts(chromosome)
a.calculate_fitness()

0

In [21]:
# 2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.

from collections import defaultdict
from itertools import combinations

class MaximizeResourceUtilization:
    def __init__(self, chromosome:list, max_groups_per_assistant, max_shift_per_assistant):
        self.chromosome = chromosome
        self.max_groups_per_assistant = max_groups_per_assistant
        self.max_shift_per_assistant = max_shift_per_assistant
        self.assistants_assignments = defaultdict(list)
        self.assistants_shifts = defaultdict(list)
        self.overload_penalty = 0

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Max Groups Per Assistant: {self.max_groups_per_assistant}, Max Shift Per Assistant: {self.max_shift_per_assistant}, Assistants Assignments: {self.assistants_assignments}, Assistants Shifts: {self.assistants_shifts}, Overload Penalty: {self.overload_penalty}"
    
    def calculate_assistants_assignments(self):
        self.assistants_assignments.clear()
        for gene in self.chromosome.genes:
            self.assistants_assignments[gene.assistant].append(gene.group)
        return self.assistants_assignments
    
    def calculate_assistants_shifts(self):
        self.assistants_shifts.clear()
        for gene in self.chromosome.genes:
            self.assistants_shifts[gene.assistant].append(gene.time_slot)
        return self.assistants_shifts
    
    def calculate_overload_penalty(self):
        self.overload_penalty = 0
        self.calculate_assistants_assignments()
        self.calculate_assistants_shifts()
        for assistant , assignments in self.assistants_assignments.items():
            assignments_penalty = max(0, len(assignments) - self.max_groups_per_assistant)
            shift_penalty = (max(0, len(self.assistants_shifts[assistant]) - self.max_shift_per_assistant))
            self.overload_penalty += assignments_penalty + shift_penalty
        return self.overload_penalty
    
    def calculate_fitness(self):
        self.calculate_overload_penalty()
        return self.overload_penalty
    
b = MaximizeResourceUtilization(chromosome, 2, 2)
b.calculate_fitness()

        

2

In [22]:
group_data.get_group_schedule(gene.group)

{'Friday': {'Shift1': False,
  'Shift2': True,
  'Shift3': False,
  'Shift4': True,
  'Shift5': False,
  'Shift6': True},
 'Monday': {'Shift1': True,
  'Shift2': True,
  'Shift3': False,
  'Shift4': False,
  'Shift5': True,
  'Shift6': False},
 'Tuesday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Saturday': {'Shift1': False,
  'Shift2': False,
  'Shift3': True,
  'Shift4': False,
  'Shift5': True,
  'Shift6': True},
 'Thursday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': True,
  'Shift6': False},
 'Wednesday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': True,
  'Shift5': True,
  'Shift6': True}}

In [23]:
group_data.get_group_schedule(gene.group)["Friday"]["Shift1"]

False

In [24]:
# 3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

class ParticipantAvailability:
    def __init__(self, chromosome, group_data: GroupData):
        self.chromosome = chromosome
        self.group_data = group_data
        self.unsatified_participants = defaultdict(list)
        self.unsatified_participants_count = 0

    def calculate_unsatisfied_participants(self):
        self.unsatified_participants.clear()
        for gene in self.chromosome.genes:
            participant_availability = group_data.get_group_schedule(gene.group)
            if not participant_availability[gene.time_slot.day][gene.time_slot.shift]:
                self.unsatified_participants[gene.group].append(gene.time_slot)
                self.unsatified_participants_count += 1
        return self.unsatified_participants_count
    
    def calculate_fitness(self):
        self.calculate_unsatisfied_participants()
        return self.unsatified_participants_count
    
c = ParticipantAvailability(chromosome, group_data)
c.calculate_fitness()
            

2

In [25]:
c.unsatified_participants

defaultdict(list,
            {1: [TimeSlot(date='2021-01-01', day='Friday', shift='Shift1')],
             3: [TimeSlot(date='2021-01-01', day='Monday', shift='Shift1')]})

In [ ]:
# 4. Maximize Weekly Chapter Coverage: Maximize the number of chapters covered in a week by ensuring that each chapter is assigned to a time slot that is available to them.

In [ ]:
module_data.get_module_chapter(gene.module_chapter)

<QuerySet [<Chapter: U-1>, <Chapter: U-2>, <Chapter: U-3>, <Chapter: U-4>, <Chapter: U-5>, <Chapter: U-6>, <Chapter: U-7>, <Chapter: U-8>]>

In [ ]:
chromosome.genes

[Gene(lab=2, module=2, module_chapter=6, group=29, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2023, 9, 29, 4, 5, 2, 158967), day='Wednesday', shift='Shift2')),
 Gene(lab=2, module=1, module_chapter=7, group=31, assistant=4, time_slot=TimeSlot(date=datetime.datetime(2023, 9, 28, 4, 5, 2, 161479), day='Saturday', shift='Shift3')),
 Gene(lab=3, module=1, module_chapter=3, group=23, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2023, 9, 23, 4, 5, 2, 164760), day='Thursday', shift='Shift4')),
 Gene(lab=3, module=1, module_chapter=8, group=10, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2023, 9, 22, 4, 5, 2, 167305), day='Monday', shift='Shift2')),
 Gene(lab=1, module=1, module_chapter=1, group=2, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2023, 9, 22, 4, 5, 2, 169330), day='Wednesday', shift='Shift2')),
 Gene(lab=4, module=1, module_chapter=7, group=8, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2023, 9, 26, 4, 5, 2, 171336), day='Thurs

#### Constants: 
1. Number of days in a week
2. Number of time slots in a day

In [28]:
class Constant:
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shifts = ["Shift1", "Shift2", "Shift3", "Shift4", "Shift5", "Shift6"]

### Population Initialization

In [29]:
# Population Representation
# 1. Chromosome
# 2. Population Size
# 3. Mutation Rate
# 4. Crossover Rate
# 5. Selection Strategy
# 6. Fitness Function

from random import random, randint, choice
from copy import deepcopy
from math import ceil

class Population:
    def __init__(self, chromosomes, population_size, mutation_rate, crossover_rate, selection_strategy, fitness_function):
        self.chromosomes = chromosomes
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.selection_strategy = selection_strategy
        self.fitness_function = fitness_function
        
    def __repr__(self):
        return f"Chromosomes: {self.chromosomes}, Population Size: {self.population_size}, Mutation Rate: {self.mutation_rate}, Crossover Rate: {self.crossover_rate}, Selection Strategy: {self.selection_strategy}, Fitness Function: {self.fitness_function}"
    
    def __str__(self) -> str:
        return f"Chromosomes: {self.chromosomes}, Population Size: {self.population_size}, Mutation Rate: {self.mutation_rate}, Crossover Rate: {self.crossover_rate}, Selection Strategy: {self.selection_strategy}, Fitness Function: {self.fitness_function}"
    
    def generate(self):
        return Population(self.chromosomes, self.population_size, self.mutation_rate, self.crossover_rate, self.selection_strategy, self.fitness_function)
    
    def calculate_fitness(self):
        for chromosome in self.chromosomes:
            chromosome.fitness = self.fitness_function(chromosome)
        return self.chromosomes
    
    def selection(self):
        return self.selection_strategy(self.chromosomes, self.population_size)
    
    def crossover(self, chromosome1, chromosome2):
        if random() < self.crossover_rate:
            genes1 = chromosome1.genes
            genes2 = chromosome2.genes
            chromosome1.genes = genes1[:len(genes1)//2] + genes2[len(genes2)//2:]
            chromosome2.genes = genes2[:len(genes2)//2] + genes1[len(genes1)//2:]
        return chromosome1, chromosome2
    
    def mutation(self, chromosome):
        if random() < self.mutation_rate:
            genes = chromosome.genes
            genes[randint(0, len(genes)-1)] = choice(self.chromosomes).genes[randint(0, len(genes)-1)]
        return chromosome
    
    def next_generation(self):
        next_generation = []
        for chromosome in self.selection():
            next_generation.append(self.mutation(chromosome))
        for chromosome1, chromosome2 in zip(next_generation[::2], next_generation[1::2]):
            self.crossover(chromosome1, chromosome2)
        return next_generation
    
    def evolve(self, generations):
        for _ in range(generations):
            self.chromosomes = self.next_generation()
            self.calculate_fitness()
        return self.chromosomes
    
    def best(self):
        return max(self.chromosomes, key=lambda chromosome: chromosome.fitness)
    
    def worst(self):
        return min(self.chromosomes, key=lambda chromosome: chromosome.fitness)
    
    def average(self):
        return sum(chromosome.fitness for chromosome in self.chromosomes) / len(self.chromosomes)
    
    def __iter__(self):
        return iter(self.chromosomes)
    
    def __getitem__(self, index):
        return self.chromosomes[index]
    
    def __len__(self):
        return len(self.chromosomes)
    
    

In [93]:

from datetime import datetime, timedelta

In [94]:
(( module_data.get_module(1).end_date-module_data.get_module(1).start_date).days + 1) // 7

10

In [125]:
Constant.days.index("Monday")

0

In [239]:
# Schedule Generator: Generate Gene, Generate Chromosome, Generate Population
# 1. Generate Gene
# 2. Generate Chromosome
# 3. Generate Population

class ScheduleGenerator:
    def __init__(self, lab_data: LaboratoryData, module_data: ModuleData, participant_data: ParticipantData, group_data: GroupData, assistant_data: AssistantData, chapter_data: ChapterData):
        self.lab_data = lab_data
        self.module_data = module_data
        self.participant_data = participant_data
        self.group_data = group_data
        self.assistant_data = assistant_data
        self.chapter_data = chapter_data
        
    def generate_gene(self):
        while True:
            # Generate gene
            laboratory = choice(self.lab_data.laboratories)["id"]
            module = choice(self.lab_data.get_module(laboratory))["id"]
            module_chapter = choice(self.module_data.get_module_chapter(module))["id"]
            group = choice(self.module_data.get_group(module))["id"]
            assistant = choice(self.lab_data.get_assistant(laboratory))["id"]

            # Generate time slot
            module_start_date = self.module_data.get_module(module).start_date
            module_end_date = self.module_data.get_module(module).end_date
            module_duration = (module_end_date - module_start_date).days + 1
            module_weeks = ceil(module_duration / 7)
            
            week = randint(0, module_weeks-1)
            day = choice(Constant.days)
            shift = choice(Constant.shifts)
            date = module_start_date + timedelta(days=week*7 + Constant.days.index(day))
            time_slot = TimeSlot(date, day, shift)
            gene = Gene(laboratory, module, module_chapter, group, assistant, time_slot)
            constraint_checker = ConstraintChecker(gene, self.lab_data, self.module_data, self.group_data, self.assistant_data, self.chapter_data)
            if constraint_checker.check():
                return gene
    
    def generate_chromosome(self, chromosome_size=10):
        genes = []
        for _ in range(chromosome_size):
            genes.append(self.generate_gene())
        return Chromosome(genes, 0)
    
    def generate_population(self, population_size):
        chromosomes = []
        for _ in range(population_size):
            chromosomes.append(self.generate_chromosome())
        return Population(chromosomes, population_size, 0.1, 0.8, self.selection_strategy, self.fitness_function)
    
    def selection_strategy(self, chromosomes, population_size):
        return sorted(chromosomes, key=lambda chromosome: chromosome.fitness, reverse=True)[:population_size]
    
    def fitness_function(self, chromosome):
        minimize_conflicts = MinimizeConflicts(chromosome)
        maximize_resource_utilization = MaximizeResourceUtilization(chromosome, 2, 2)
        participant_availability = ParticipantAvailability(chromosome, self.group_data)
        minimize_conflicts.calculate_fitness()
        maximize_resource_utilization.calculate_fitness()
        participant_availability.calculate_fitness()
        return 1 / (minimize_conflicts.fitness + maximize_resource_utilization.fitness + participant_availability.fitness)
    
    def generate(self, population_size, generations):
        population = self.generate_population(population_size)
        population.calculate_fitness()

        for _ in range(generations):
            population = population.next_generation()
            population.calculate_fitness()  
        return population.best()
    
    def generate_schedule(self, chromosome):
        schedule = {}
        for gene in chromosome.genes:
            if gene.lab not in schedule:
                schedule[gene.lab] = {}
            if gene.time_slot.day not in schedule[gene.lab]:
                schedule[gene.lab][gene.time_slot.day] = {}
            schedule[gene.lab][gene.time_slot.day][gene.time_slot.shift] = gene.group
        return schedule
    
    def generate_schedule_df(self, chromosome):
        schedule = self.generate_schedule(chromosome)
        return pd.DataFrame(schedule).fillna("-")
    
    

In [240]:
chromosome = ScheduleGenerator(lab_data, module_data, participant_data, group_data, assistant_data, chapter_data).generate_chromosome(1000)
chromosome.genes

[Gene(lab=1, module=1, module_chapter=3, group=36, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 5, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Saturday', shift='Shift4')),
 Gene(lab=1, module=1, module_chapter=6, group=34, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 28, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift3')),
 Gene(lab=1, module=1, module_chapter=5, group=32, assistant=6, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 13, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Tuesday', shift='Shift6')),
 Gene(lab=1, module=1, module_chapter=4, group=36, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 23, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Wednesday', shift='Shift6')),
 Gene(lab=1, module=1, module_chapter=7, group=10, assistant=6, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 9, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Wednesday', shift='

Gene(lab=1, module=1, module_chapter=3, group=12, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 31, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Monday', shift='Shift5'))